In [1]:
import bs4 as bs
import pickle
import requests
import pandas as pd
import json
from dateutil import parser
from datetime import datetime
import os
import time
from pandas.io.json import json_normalize
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
import wrds
from datetime import datetime as dt
import numpy as np
import glob, os
import csv

pd.set_option("display.precision", 8)

#AlphaVantage API Key
key = 'G6MVTXA5Z8N41LFL'

## Tickers

> ### Gets currently listed SP500 companies and thier CIKs

In [2]:
# gets list of SP500 companies and their CIKs

#standard: pull from yahoo finance, yfinance
#

def save_SP500_tickers():
    resp = requests.get('https://web.archive.org/web/20200411084847/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    ciks = []
    
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text[:-1]
        cik = row.findAll('td')[7].text[:]
        
        tickers.append(ticker)
        ciks.append(cik)
    
    #drops '\n' from certain entries in CIK list
    ciks = list(map(lambda x:x.strip(),ciks))
    
    tickers_ciks_df = pd.DataFrame({'ticker': tickers, 'cik': ciks})

        
    return tickers_ciks_df

> ### Prepping tickers list

In [3]:
tickers_ciks= save_SP500_tickers()

In [4]:
tickers = tickers_ciks['ticker'] 
tickers = [item.replace('-','.') for item in tickers]
tickers = [item.replace('.B','-A') for item in tickers]
ciks = tickers_ciks['cik']

> ### Remove tickers from list if change detected in change list

In [5]:
changes = pd.read_excel('sp500change.xlsx')

In [6]:
changes['Ticker'] = changes['Ticker'].astype(str)

In [7]:
changes['Ticker'] = [item.replace('-','.') for item in changes['Ticker']]
changes['Ticker'] = [item.replace('.B','-A') for item in changes['Ticker']]

In [8]:
tickers = [x for x in tickers if x not in list(changes['Ticker'])]

## Financial Modelling Company Quarterly Financials

> ### Get Income Statements and Balance Sheets

In [9]:
fpc_key = '7997a851d755732d0789c013afdfbfea'
n_months = 58

In [10]:
fundamentals_start = pd.to_datetime('2009-02-09', format = '%Y-%m-%d')
fundamentals_end = pd.to_datetime('2020-02-03', format = '%Y-%m-%d')

In [11]:
'''
income_dfs = []
for company in consistent_companies_dfs:
    r = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{company.index.name}?period=quarter&limit={n_months}&apikey={fpc_key}")
    r= r.json()
    data = pd.DataFrame.from_dict(r)
    income_dfs.append(data)
#'''

'\nincome_dfs = []\nfor company in consistent_companies_dfs:\n    r = requests.get(f"https://financialmodelingprep.com/api/v3/income-statement/{company.index.name}?period=quarter&limit={n_months}&apikey={fpc_key}")\n    r= r.json()\n    data = pd.DataFrame.from_dict(r)\n    income_dfs.append(data)\n#'

In [12]:
'''
balance_dfs = []
for company in consistent_companies_dfs:
    r = requests.get(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company.index.name}?period=quarter&limit={n_months}&apikey={fpc_key}")
    r= r.json()
    data = pd.DataFrame.from_dict(r)
    balance_dfs.append(data)
#'''

'\nbalance_dfs = []\nfor company in consistent_companies_dfs:\n    r = requests.get(f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company.index.name}?period=quarter&limit={n_months}&apikey={fpc_key}")\n    r= r.json()\n    data = pd.DataFrame.from_dict(r)\n    balance_dfs.append(data)\n#'

> ### Dump or Load Quarterly Data to Pickle

In [13]:
#dump to pickle
'''
with open('quarterly_income', 'wb') as fh:
    pickle.dump(income_dfs, fh)
    
with open('quarterly_balance', 'wb') as fh:
    pickle.dump(balance_dfs, fh)
'''

"\nwith open('quarterly_income', 'wb') as fh:\n    pickle.dump(income_dfs, fh)\n    \nwith open('quarterly_balance', 'wb') as fh:\n    pickle.dump(balance_dfs, fh)\n"

In [14]:
# load from pickle

#'''
with open("quarterly_income","rb") as fin:
    income_dfs = pickle.load(fin)


with open("quarterly_balance","rb") as fin:
    balance_dfs = pickle.load(fin)
#'''


> ### Get Quarterly Net Income, Revenue and Total Assets 

> > #### Cleaned Income Statement Data

In [15]:
clean_income_dfs = []
cols_to_keep = ['date', 'symbol', 'reportedCurrency', 
                    'fillingDate','period', 'revenue','netIncome']
for company in income_dfs:
    company = company[cols_to_keep]
    company = company.set_index('date')
    company = company.iloc[::-1]
    clean_income_dfs.append(company)

> > #### Cleaned Income Statement Data

In [16]:
clean_balance_dfs = []
cols_to_keep = ['date', 'symbol', 'reportedCurrency', 
                    'fillingDate','period', 'totalAssets']
for company in balance_dfs:
    company = company[cols_to_keep]
    company = company.set_index('date')
    company = company.iloc[::-1]
    clean_balance_dfs.append(company)

### > ### Drop Companies with Missing Data

> > #### Balance Sheet

In [17]:

balance_not_dropped = []


for df in clean_balance_dfs:
    df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
    df['difference'] = df.index.to_series().diff().dt.days
    balance_not_dropped.append(pd.DataFrame(df))
   


In [18]:


clean_balance_dfs_not_dropped = []

for df in balance_not_dropped:
    if df['difference'].bfill().min() > 75:
        df['fillingDate'] = pd.to_datetime(df['fillingDate'], format='%Y-%m-%d', errors='ignore')
        clean_balance_dfs_not_dropped.append(df[:-1])
        
        

In [19]:
#clean_balance_dfs_not_dropped.insert(0, clean_balance_dfs_not_dropped.pop(167))

balance_date_matched = []

#clean_balance_dfs.sort(key=lambda df: df.index.min())

for count, df in enumerate(clean_balance_dfs_not_dropped):

        
    if count>0 and count < len(clean_balance_dfs_not_dropped)-1:
        
        
        merged_dataframe = pd.merge_asof(clean_balance_dfs_not_dropped[0],
                                         clean_balance_dfs_not_dropped[count+1],
                                        on='date')
        balance = merged_dataframe.iloc[:, np.r_[0,7:12]]
        balance = balance.set_index('date')
        balance = balance.rename(columns = lambda x : str(x)[:-2]) 
        balance_date_matched.append(balance)

        balance = []
        merged_dataframe = []
        
    if count == len(clean_balance_dfs_not_dropped)-1:

        merged_dataframe = pd.merge_asof(clean_balance_dfs_not_dropped[0],
                                         clean_balance_dfs_not_dropped[count],
                                        on='date')
        balance = merged_dataframe.iloc[:, np.r_[0,7:12]]
        balance = balance.set_index('date')
        balance = balance.rename(columns = lambda x : str(x)[:-2]) 
        balance_date_matched.append(balance)

        #balance = []
        #merged_dataframe = []


    

In [20]:
trimmed_balance = []
for df in balance_date_matched:
    x = df["2009-09-01":"2019-10-15"]
    
    if not x.isna().any().any():
        trimmed_balance.append(x)
        
trimmed_balance = trimmed_balance[:-1]

In [21]:
for df in trimmed_balance:
    print(df.iloc[0,0])

ADM
ADBE
ADP
AES
A
APD
ALL
GOOGL
MO
AMZN
AEE
AEP
AIG
AMP
ABC
AMGN
ADI
APA
AAPL
AMAT
T
ADSK
AZO
AVY
BALL
BAC
BBWI
BAX
BDX
BRK-A
BIO
TECH
BIIB
BK
BA
BXP
BSX
BMY
BRO
CPT
CPB
COF
CAH
CCL
CAT
CNP
CRL
CVX
CB
CI
CINF
CTAS
CSCO
CLX
CMS
KO
CL
CMCSA
CMA
CAG
COP
ED
STZ
GLW
CSGP
CTRA
CSX
CMI
CVS
DHI
DHR
DRI
DE
DVN
DXCM
DIS
D
DPZ
DOV
DTE
DUK
EMN
EBAY
ECL
EIX
EA
ELV
LLY
EMR
ETR
EOG
EFX
EQR
EL
EXC
XOM
FDS
FDX
FITB
FE
FISV
F
BEN
FCX
GD
GIS
GPC
GILD
GS
HAL
HIG
HAS
HSY
HES
HD
HPQ
HUM
HBAN
IBM
ITW
IP
IPG
IFF
INTU
JNJ
JCI
JPM
JNPR
K
KEY
KMB
KIM
KLAC
KR
LH
LEN
LNC
LOW
LUMN
MRO
MAR
MMC
MAS
MKC
MCD
MCK
MDT
MRK
MU
MSFT
MOH
TAP
MPWR
MCO
MS
NTAP
NWL
NEM
NEE
NKE
NI
NDSN
NSC
NTRS
NUE
NVDA
OXY
OMC
ON
ORCL
PCAR
PARA
PH
PAYX
PEP
PKI
PFE
PNW
PNC
POOL
PPG
PPL
PFG
PG
PGR
PRU
PEG
PTC
PSA
PHM
DGX
ROK
SEE
SRE
SHW
SBNY
SPG
SNA
SO
LUV
SWK
SBUX
STT
SYK
SYY
TROW
TGT
TDY
TXN
TXT
TMO
TJX
TRMB
TYL
TSN
USB
UNP
UPS
UNH
VLO
VRSN
VZ
VFC
VNO
VMC
WMT
WBD
WM
WAT
WFC
WST
WY
WHR
WMB
GWW
XEL
YUM
ZION


> > #### Income Statement

In [22]:
income_not_dropped = []

for df in clean_income_dfs:
    df.index = pd.to_datetime(df.index, format = '%Y-%m-%d')
    df['difference'] = df.index.to_series().diff().dt.days
    income_not_dropped.append(pd.DataFrame(df))

In [23]:
clean_income_dfs_not_dropped = []

for df in income_not_dropped:
    if df['difference'].bfill().min() > 75:
        df['fillingDate'] = pd.to_datetime(df['fillingDate'], format='%Y-%m-%d', errors='ignore')
        clean_income_dfs_not_dropped.append(df[:-1])

In [24]:
#clean_balance_dfs_not_dropped.insert(0, clean_balance_dfs_not_dropped.pop(167))

income_date_matched = []

#clean_balance_dfs.sort(key=lambda df: df.index.min())

for count, df in enumerate(clean_income_dfs_not_dropped):

        
    if count>0 and count < len(clean_income_dfs_not_dropped)-1:
        
        
        merged_dataframe = pd.merge_asof(clean_income_dfs_not_dropped[0],
                                         clean_income_dfs_not_dropped[count+1],
                                        on='date')
        income = merged_dataframe.iloc[:, np.r_[0,8:14]]
        income = income.set_index('date')
        income = income.rename(columns = lambda x : str(x)[:-2]) 
        income_date_matched.append(income)

        income = []
        merged_dataframe = []
        
    if count == len(clean_income_dfs_not_dropped)-1:

        merged_dataframe = pd.merge_asof(clean_income_dfs_not_dropped[0],
                                         clean_income_dfs_not_dropped[count],
                                        on='date')
        income = merged_dataframe.iloc[:, np.r_[0,6:14]]
        income = income.set_index('date')
        income = income.rename(columns = lambda x : str(x)[:-2]) 
        income_date_matched.append(income)

        #income = []
        #merged_dataframe = []


    

In [25]:
trimmed_income = []
for df in income_date_matched:
    x = df["2009-09-01":"2019-10-15"]
    #x.netIncome.values.astype(int)
    if not x.isna().any().any():
        trimmed_income.append(x)
trimmed_income = trimmed_income[:-1]

In [26]:
trimmed_balance[0]

,symbol,reportedCurrency,fillingDate,period,totalAssets
date,,,,,
2009-09-30,ADM,USD,2009-09-30,Q3,31861000000
2009-12-31,ADM,USD,2009-12-31,Q4,33529000000
2010-03-31,ADM,USD,2010-03-31,Q1,32326000000
2010-06-30,ADM,USD,2010-06-30,Q2,31548000000
2010-09-30,ADM,USD,2010-11-08,Q3,36036000000
2010-12-31,ADM,USD,2011-02-09,Q4,42610000000
2011-03-31,ADM,USD,2011-05-10,Q1,43763000000
2011-06-30,ADM,USD,2011-08-25,Q2,42193000000
2011-09-30,ADM,USD,2011-09-30,Q3,41448000000


In [27]:
for df in trimmed_balance:
    print(df.shape)

(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)
(41, 5)


> ### Calculate Return on Assets and Asset Turnover

In [28]:
ratio_dfs = []

for company, c in enumerate(trimmed_balance):
    


    merged = pd.merge(trimmed_income[int(company)], trimmed_balance[int(company)],
                          on=['date', 'symbol', 'reportedCurrency','fillingDate', 'period'])
    ratios = pd.DataFrame()
    ratios['RoA'] = merged[['netIncome']].div(merged.totalAssets, axis = 0)
    ratios['AT'] = merged[['revenue']].div(merged.totalAssets, axis = 0)
    
    #reverse dataframes so date go in ascending order
    #merged = merged.iloc[::-1]
    
    df = np.concatenate([ratios.RoA.values,ratios.AT.values])
    df = pd.DataFrame(df, columns = [c.iloc[0]['symbol']])
    #df = df.set
        #df.columns = c.iloc[0]['symbol']
    ratio_dfs.append(df)

In [29]:
ratio_dfs[0]

,ADM
0,0.01556762
1,0.01691073
2,0.01302357
3,0.01413719
4,0.00957376
...,...
77,0.39834611
78,0.39054196
79,0.35466154
80,0.38241506


> ### Format Ratios Data 

In [35]:
#final list of ratio dfs.
#RoA and AT are concatenated together vertically as a single list. 
#41 quarters of data for each company
ratios_final =[]
for df in ratio_dfs:
    
    if df.shape[0] == 82:
        ratios_final.append(df)

> ### Export Ratios as CSV

In [44]:
ratios_export = pd.concat(ratios_final, axis=1).T.dropna(how='all',axis=0).dropna(how='all',axis=1)

In [45]:
ratios_export.shape

(235, 82)

In [46]:
ratios_export.replace([np.inf, -np.inf], np.nan, inplace=True)

In [48]:
ratios_export.dropna(how="any", inplace=True)

In [49]:
ratios_export.shape

(234, 82)

In [51]:
ratios_export[:-1].to_csv('Export/ratios.csv')